In [1]:
%matplotlib inline

seed_value = 100
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# Data uploading

In [2]:
#AIRPORT_ICAO = "ESGG"
AIRPORT_ICAO = "ESSA"

DATA_DIR = os.path.join("..", "..")
DATA_DIR = os.path.join(DATA_DIR, "data")
DATA_DIR = os.path.join(DATA_DIR, "Regression")

# WIF
filename = AIRPORT_ICAO + "_metrics_WIF_vertical_PIs_by_hour.csv"
full_filename = os.path.join(DATA_DIR, filename)
metrics_WIF_vertical_PIs_by_hour_df = pd.read_csv(full_filename, sep=' ')

# TIF
filename = AIRPORT_ICAO + "_metrics_TIF_vertical_PIs_by_hour.csv"
full_filename = os.path.join(DATA_DIR, filename)
metrics_TIF_vertical_PIs_by_hour_df = pd.read_csv(full_filename, sep=' ')

# Average Time On Levels

## Medians of Average Time On Levels vs TIF

                            OLS Regression Results                            
Dep. Variable:     timeOnLevelsMedian   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     13.42
Date:                Thu, 15 Sep 2022   Prob (F-statistic):            0.00637
Time:                        16:28:42   Log-Likelihood:                -2.1550
No. Observations:                  10   AIC:                             8.310
Df Residuals:                       8   BIC:                             8.915
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.8372      0.229     16.738      0.0

In [3]:
df = metrics_TIF_vertical_PIs_by_hour_df.copy()
df['TIF'] = df['TIF'].astype(float)
#df = df[df['numberOfFlights']>0]
#p1 = df["timeOnLevelsPercentMean"].quantile(0.05)
#p2 = df["timeOnLevelsPercentMean"].quantile(0.95)
#df = df.loc[(df['timeOnLevelsPercentMean'] > p1) & (df['timeOnLevelsPercentMean'] < p2) ]

df.set_index(['TIF'], inplace=True)
df.sort_index(inplace=True)

median_df = pd.DataFrame(columns=['TIF', 'timeOnLevelsMedian'])

for idx, group in df.groupby(level='TIF'):
    
    idx_df = group['timeOnLevelsPercentMean']

    idx_median = np.median(idx_df)
    median_df = median_df.append({'TIF': idx, 'timeOnLevelsMedian': idx_median}, ignore_index=True)

median_df['TIF'] = median_df['TIF'].astype(float)
median_df['timeOnLevelsMedian'] = median_df['timeOnLevelsMedian'].astype(float)

sns.set(rc={'figure.figsize':(1,1)})
sns_plot = sns.lmplot(x='TIF',y='timeOnLevelsMedian',data=median_df,fit_reg=True, height=4, aspect=3/2);
plt.xlabel('Traffic Impact Factor', fontsize=15)
plt.ylabel('Time Flown Level Median [%]', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
fig_name = "TimeOnLevelsVsTIF.png"
plt.savefig(fig_name, bbox_inches='tight')

#statsmodel:
X_df = median_df[['TIF']]
y_df = median_df[['timeOnLevelsMedian']]

X2 = sm.add_constant(X_df.values)
est = sm.OLS(y_df.astype(float), X2.astype(float))
est2 = est.fit()
print(est2.summary()) 
# with outliers - ESGG: 0.13 ESSA: 0.63
# without outliers - ESGG: 0.01 ESSA: 0.4

x = X_df['TIF'].values.tolist()
y = y_df['timeOnLevelsMedian'].values.tolist()

# calculate pearnon's correlation
pearsonr, p = stats.pearsonr(x,y) 
print(pearsonr, p)

# calculate spearman's correlation
spearmanr, p = stats.spearmanr(x, y)
print(spearmanr, p)

## Medians of Average Time On Levels vs WIF

                            OLS Regression Results                            
Dep. Variable:     timeOnLevelsMedian   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     20.93
Date:                Thu, 15 Sep 2022   Prob (F-statistic):            0.00181
Time:                        16:28:43   Log-Likelihood:                -7.6936
No. Observations:                  10   AIC:                             19.39
Df Residuals:                       8   BIC:                             19.99
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.5338      0.399      8.859      0.0

In [4]:
df = metrics_WIF_vertical_PIs_by_hour_df.copy()
#df = df[df['numberOfFlights']>0]
p1 = df["timeOnLevelsPercentMean"].quantile(0.05)
p2 = df["timeOnLevelsPercentMean"].quantile(0.95)
df = df.loc[(df['timeOnLevelsPercentMean'] > p1) & (df['timeOnLevelsPercentMean'] < p2) ]

df.set_index(['WIF'], inplace=True)
df.sort_index(inplace=True)

median_df = pd.DataFrame(columns=['WIF', 'timeOnLevelsMedian'])

for idx, group in df.groupby(level='WIF'):
    
    idx_df = group['timeOnLevelsPercentMean']
    
    idx_median = np.median(idx_df)
    median_df = median_df.append({'WIF': idx, 'timeOnLevelsMedian': idx_median}, ignore_index=True)

median_df['WIF'] = median_df['WIF'].astype(float)
median_df['timeOnLevelsMedian'] = median_df['timeOnLevelsMedian'].astype(float)

sns.set(rc={'figure.figsize':(1,1)})

sns.lmplot(x='WIF',y='timeOnLevelsMedian',data=median_df,fit_reg=True, height=4, aspect=3/2);
plt.xlabel('Weather Impact Factor', fontsize=15)
plt.ylabel('Time Flown Level Median [%]', fontsize=15)
ticks = [i for i in range(1, 11)]
plt.xticks(ticks, fontsize=15)
plt.yticks(fontsize=15)
#plt.text(2.2, 6.2, '$R^2 = 0.73$', fontsize = 22)
plt.text(2.2, 7.1, '$R^2 = 0.93$', fontsize = 22)

#statsmodel:
X_df = median_df[['WIF']]
y_df = median_df[['timeOnLevelsMedian']]

X2 = sm.add_constant(X_df.values)
est = sm.OLS(y_df, X2)
est2 = est.fit()
print(est2.summary()) 
# with outliers - ESGG: 0.57 ESSA: 0.76
# without outliers - ESGG: 0.93 ESSA: 0.73

x = X_df['WIF'].values.tolist()
y = y_df['timeOnLevelsMedian'].values.tolist()

# calculate pearnon's correlation
pearsonr, p = stats.pearsonr(x,y) 
print(pearsonr, p)

# calculate spearman's correlation
spearmanr, p = stats.spearmanr(x, y)
print(spearmanr, p)